# Timelag Maps: Observed
Compute timelags from observed AIA data and save timelags and peak cross-correlations to FITS files for each channel pair.

In [ ]:
import os
import sys

import numpy as np
import distributed
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn
from sunpy.map import Map,GenericMap
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.utils.console import ProgressBar

import synthesizAR
from synthesizAR.instruments import InstrumentSDOAIA
from synthesizAR.analysis import DistributedAIACube,AIATimelags

%matplotlib inline

In [ ]:
cluster = distributed.LocalCluster(n_workers=16,threads_per_worker=4,)
client = distributed.Client(cluster)
client

In [ ]:
channels = [94,131,171,193,211,335]

In [ ]:
channel_pairs = [(94,335),
                 (94,171),
                 (94,193),
                 (94,131),
                 (94,211),
                 (335,131),
                 (335,193),
                 (335,211), 
                 (335,171),
                 (211,131),
                 (211,171),
                 (211,193),
                 (193,171),
                 (193,131),
                 (171,131),]

In [ ]:
tl = AIATimelags(*[DistributedAIACube.from_files(
    f'/storage-home/w/wtb2/data/timelag_synthesis_v2/observational_data/aia/cutouts/aia_lev1.5_*_{c}_cutout.fits')
                   for c in channels])

In [ ]:
timelag_bounds = (-6*u.hour,6*u.hour)

In [ ]:
save_format = '../paper/data/observations/{}_{}_{}.fits'

In [ ]:
chunks = (tl[0].shape[1]//5, tl[1].shape[2]//5)

In [ ]:
for ca,cb in channel_pairs:
    timelag_map = tl.timelag_map(f'{ca}', f'{cb}', timelag_bounds=timelag_bounds, chunks=chunks)
    timelag_map.save(save_format.format('timelag', ca, cb))
    correlation_map = tl.peak_cross_correlation_map(f'{ca}', f'{cb}', timelag_bounds=timelag_bounds, chunks=chunks)
    correlation_map.save(save_format.format('correlation', ca, cb))